In [1]:
# %pip install pyigd
from pygrgl import GRG, load_immutable_grg, get_dfs_order, TraversalDirection, get_topo_order
from pygrgl.display import grg_to_cyto
from IPython.display import display, SVG
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import pyigd
from collections import defaultdict
# import torch

In [2]:
igd_file = "/global/homes/l/landrum/data/Filtered_Genotype_Data_2022.igd"
igd = pyigd.IGDFile(igd_file)

index_to_pedigree = igd.get_individual_ids()
pedigree_to_index = {pedigree: index for index, pedigree in enumerate(index_to_pedigree)}
len(pedigree_to_index)

431

In [3]:
# import the latent phenotypes
labels = pd.read_csv('~/data/labels_latentspace32_2022.csv')
labels

,Unnamed: 0,X.3,X.2,X.1,X,file_name,date,plot,date_string,yield,...,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32
0,1,1,1,1,0,06202022_136.png,0,136,6202022,194.56,...,0.535505,0.452575,0.981067,1.149227,0.995027,1.241586,0.745727,0.275623,1.029122,1.341230
1,2,2,2,2,1,06202022_87.png,0,87,6202022,187.82,...,0.509663,0.608769,1.048242,1.051819,0.925715,1.236904,0.797817,0.367011,1.078613,1.332783
2,3,3,3,3,2,09302022_474.png,9,474,9302022,197.15,...,0.445502,0.289757,0.670552,0.759065,0.643423,1.057338,0.569775,0.331227,0.924499,1.166521
3,4,4,4,4,3,07062022_269.png,2,269,7062022,155.84,...,0.768605,0.872622,1.120786,0.968971,1.417331,1.109956,1.348742,0.915474,1.201259,0.920072
4,5,5,5,5,4,09082022_129.png,7,129,9082022,171.77,...,1.657700,1.417696,0.844563,1.178993,1.399463,0.865573,1.372088,1.742877,1.372035,0.683685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,5996,5996,5996,5996,5995,09202022_349.png,8,349,9202022,157.16,...,0.633788,0.791538,0.886973,1.095859,1.292476,1.182523,1.122651,0.610794,1.108877,0.939892
5996,5997,5997,5997,5997,5996,07062022_5.png,2,5,7062022,191.35,...,0.881702,0.962112,1.045038,1.056591,1.264617,1.120266,1.072492,0.939718,1.130858,1.015790
5997,5998,5998,5998,5998,5997,07062022_20.png,2,20,7062022,162.37,...,0.934758,1.000152,1.038232,1.017878,1.407997,1.110500,1.284906,1.099208,1.225952,0.937268
5998,5999,5999,5999,5999,5998,10122022_413.png,11,413,10122022,184.29,...,0.312065,0.321582,0.896917,1.131446,0.770082,1.133601,0.710439,0.256898,1.096045,1.515961


In [4]:
grg = load_immutable_grg('/global/homes/l/landrum/data/corn.igd.final.grg')
print(f"nodes: {grg.num_nodes:,}, edges: {grg.num_edges:,}, samples: {grg.num_samples:,}")
# display(grg_to_cyto(grg))

nodes: 157,605, edges: 8,837,817, samples: 862


In [5]:
train_pedigrees = labels[labels['train_set'] == True]['Pedigree'].unique()
len(train_pedigrees)

312

In [13]:
pedigree_day_embeddings = np.zeros((len(index_to_pedigree), 12, 32), dtype=np.float32)

for i, pedigree in enumerate(index_to_pedigree):
    for day in range(12):
        for row in labels[(labels['Pedigree'] == pedigree) & (labels["date"] == day)].iterrows():
            vector = np.zeros(32, dtype=np.float32)
            for j in range(32):
                vector[j] = row[1][f"V{j+1}"]
            train_embeddings_day[day][pedigree].append(vector)
            
        
train_embeddings_day

[defaultdict(list,
             {'GEMN-0096_PHK76_0010/LH244': [array([0.2828292 , 0.44094345, 0.7879696 , 0.26604187, 1.1177627 ,
                      0.78348535, 1.1539354 , 0.3931202 , 0.91575843, 0.7445815 ,
                      0.49194172, 0.44113696, 0.8357714 , 0.9172161 , 0.78713393,
                      0.8724424 , 0.23023629, 1.2443917 , 0.47209385, 0.71157056,
                      1.2670003 , 1.2662157 , 0.42101622, 0.61248046, 1.0331635 ,
                      1.06086   , 0.9282298 , 1.1554987 , 0.9106342 , 0.29352584,
                      1.1091552 , 1.2786803 ], dtype=float32)],
              'GEMN-0225_PHP02_0003/LH244': [array([0.35412586, 0.4923315 , 0.7788141 , 0.33135307, 1.00773   ,
                      0.8676615 , 1.1362342 , 0.4469763 , 0.9668619 , 0.76286167,
                      0.55817026, 0.48073104, 0.8577914 , 0.89148283, 0.7686569 ,
                      0.92226106, 0.26983508, 1.2377924 , 0.5850061 , 0.65476924,
                      1.1920899 , 1.3

In [7]:
embedding_centroid = np.mean(train_embeddings_mean, axis=0)
embedding_centroid

array([1.0140063 , 1.0254078 , 0.85359114, 1.0515696 , 1.0068471 ,
       1.2179557 , 1.209061  , 1.0411824 , 1.2985035 , 1.1061282 ,
       1.0560567 , 1.1061872 , 1.1302127 , 0.99411744, 1.024132  ,
       1.1439859 , 0.82851946, 1.3382345 , 1.0231569 , 0.889686  ,
       1.2278976 , 1.2079741 , 0.9989247 , 0.9684895 , 1.0110663 ,
       1.1546423 , 1.1547223 , 1.0638565 , 1.1302685 , 0.9238217 ,
       1.1830806 , 1.0569884 ], dtype=float32)

In [8]:
normalized = train_embeddings_mean - embedding_centroid
normalized

array([[-0.17975003, -0.20312554, -0.15496445, ..., -0.15572894,
        -0.03912234,  0.01123059],
       [ 0.14413857,  0.15581906,  0.12757754, ...,  0.1734513 ,
         0.03034532, -0.05382264],
       [ 0.13197315,  0.17112124,  0.14257002, ...,  0.08693802,
         0.05506313, -0.0333159 ],
       ...,
       [ 0.1146903 ,  0.12170362,  0.10763818, ...,  0.09044456,
         0.00385451,  0.02199113],
       [-0.00519013,  0.00261736, -0.01983953, ..., -0.00221705,
        -0.02731586,  0.00390041],
       [ 0.04717755,  0.02932727,  0.05886918, ...,  0.01189488,
         0.06440687,  0.05641806]], dtype=float32)

In [9]:
len(list(get_topo_order(grg, TraversalDirection.UP, [0])))

51179